###### Import libraries

In [1]:
import os,sys,py3dep,xarray,rasterio,folium,geopandas,math,numpy,datetime,shutil,pynhd,shapely,matplotlib,branca,shapely,whitebox_workflows,pygeohydro,multiprocessing

###### Import hydroframe library and register pin -- see https://hydroframe.org/hydrodata

In [2]:
import hf_hydrodata as hf
hf.register_api_pin("evenson.grey@epa.gov", "2234")

###### Set run specific variables
###### NOTE: pysdaDir must be set as path to pysda clone -- see https://github.com/ncss-tech/pysda.git

In [3]:
pysdaDir = r'C:\Users\GEVENSON\GitHub\pysda'
ProjectDirectory = r'C:\Users\GEVENSON\Projects\nonperennial_stream_poc\BUCKHORN_CREEK'
DEMfname = r'C:\Users\GEVENSON\Projects\nonperennial_stream_poc\dem_preprocessing\BUCKHORN_CREEK_2M_DEM.tiff'
start_date = "2002-10-01"
end_date = "2006-09-30"
lat = 37.44870490000000 # lat, lon info: https://www.waterqualitydata.us/provider/NWIS/USGS-KY/USGS-03278100/
lon = -83.1954512000000
OVERWRITE_FLAG = False

###### Hard code Parflow CONUS1 grid info (see https://hf-hydrodata.readthedocs.io/en/latest/available_grids.html)

In [4]:
conus1_proj = '+proj=lcc +lat_1=33 +lat_2=45 +lon_0=-96.0 +lat_0=39 +a=6378137.0 +b=6356752.31'
conus1_spatext = tuple([-121.47939483437318, 31.651836025255015, -76.09875469594509, 50.49802132270979])
conus1_transform = rasterio.transform.Affine(1000.0,0.0,-1885055.4995,0.0,1000.0,-604957.0654)

###### Hard code transmissivity decay factor from Table 1 of Zhang et al. (2016) https://doi.org/10.5194/bg-13-1387-2016

In [5]:
transmissivity_decay_factor = {'clay heavy':3.2,'silty clay':3.1,'clay':2.8,'silty clay loam':2.9,'clay loam': 2.7,
                               'silt':3.4,'silt loam':2.6,'sandy clay':2.5,'loam':2.5,'sandy clay loam':2.4,
                               'sandy loam':2.3,'loamy sand':2.2,'sand':2.1,'organic':2.5}

###### Create directory architecture

In [6]:
ProjectDataDirectory            = os.path.join(ProjectDirectory,'Data')
ProjectWTDDataDirectory         = os.path.join(ProjectDataDirectory,'WTD')
ProjectWTDRawDataDirectory      = os.path.join(ProjectWTDDataDirectory,'raw')
ProjectWTDBilinearDataDirectory = os.path.join(ProjectWTDDataDirectory,'resampled_bilinear')
ProjectWTDNearestDataDirectory  = os.path.join(ProjectWTDDataDirectory,'resampled_nearest')
ProjectDEMDataDirectory         = os.path.join(ProjectDataDirectory,'DEM')
ProjectTWIDataDirectory         = os.path.join(ProjectDataDirectory,'TWI')
ProjectSoilsDataDirectory       = os.path.join(ProjectDataDirectory,'Soils')
ProjectOutputDirectory          = os.path.join(ProjectDirectory,'Output')
if not os.path.isdir(ProjectDirectory)               : os.mkdir(ProjectDirectory)
if not os.path.isdir(ProjectDirectory)               : os.mkdir(ProjectDirectory)
if not os.path.isdir(ProjectDataDirectory)           : os.mkdir(ProjectDataDirectory)
if not os.path.isdir(ProjectWTDDataDirectory)        : os.mkdir(ProjectWTDDataDirectory)
if not os.path.isdir(ProjectWTDRawDataDirectory)     : os.mkdir(ProjectWTDRawDataDirectory)
if not os.path.isdir(ProjectWTDBilinearDataDirectory): os.mkdir(ProjectWTDBilinearDataDirectory)
if not os.path.isdir(ProjectWTDNearestDataDirectory) : os.mkdir(ProjectWTDNearestDataDirectory)
if not os.path.isdir(ProjectDEMDataDirectory)        : os.mkdir(ProjectDEMDataDirectory)
if not os.path.isdir(ProjectTWIDataDirectory)        : os.mkdir(ProjectTWIDataDirectory)
if not os.path.isdir(ProjectSoilsDataDirectory)      : os.mkdir(ProjectSoilsDataDirectory)
if not os.path.isdir(ProjectOutputDirectory)         : os.mkdir(ProjectOutputDirectory)

###### Set datetime dimension

In [7]:
start_date_split = start_date.split('-')
start_datetime = datetime.datetime(year=int(start_date_split[0]),month=int(start_date_split[1]),day=int(start_date_split[2]))
end_date_split = end_date.split('-')
end_datetime = datetime.datetime(year=int(end_date_split[0]),month=int(end_date_split[1]),day=int(end_date_split[2]))
datetime_dim = list()
idatetime = start_datetime
while idatetime <= end_datetime:
    datetime_dim.append(idatetime)
    idatetime += datetime.timedelta(days=1)
datetime_dim = numpy.array(datetime_dim)

###### Identify HUC/domain

In [8]:
huc_level = 'huc12' 
idp = shapely.geometry.Point(lon, lat)
wbdbasins = pygeohydro.WBD(huc_level) 
wbdbasins  = wbdbasins.bygeom(idp)
fname_domain = os.path.join(ProjectDataDirectory,'domain.gpkg')
domain = wbdbasins.dissolve()
domain.to_file(fname_domain, driver="GPKG")
del wbdbasins
del idp

###### Get domain lat/lon bounding box

In [9]:
buffered_domain = domain.to_crs(conus1_proj).buffer(distance=2000).to_crs("EPSG:4326")
lat_min = float(buffered_domain.bounds['miny'].iloc[0])
lat_max = float(buffered_domain.bounds['maxy'].iloc[0])
lon_min = float(buffered_domain.bounds['minx'].iloc[0])
lon_max = float(buffered_domain.bounds['maxx'].iloc[0])

###### Get domain bounding box for Parflow grid

In [10]:
grid_minx, grid_miny = hf.from_latlon("conus1", lat_min, lon_min)
grid_maxx, grid_maxy = hf.from_latlon("conus1", lat_max, lon_max)
grid_minx = math.floor(grid_minx)
grid_maxx = math.ceil(grid_maxx)
grid_miny = math.floor(grid_miny)
grid_maxy = math.ceil(grid_maxy)

###### Reset lat/lon bounding box to include border Parflow grid cells

In [11]:
latlon_bounds = hf.to_latlon("conus1", *[grid_minx, grid_miny, grid_maxx, grid_maxy])
lon_min = latlon_bounds[1]
lat_min = latlon_bounds[0]
lon_max = latlon_bounds[3]
lat_max = latlon_bounds[2]

###### Download the Parflow water table depth data (if not already downloaded)

In [12]:
download_flag = False
fnames_wtd = list()
for idatetime in datetime_dim:
    fname_wtd = os.path.join(ProjectWTDRawDataDirectory,'wtd_'+idatetime.strftime('%Y%m%d')+'.tiff')
    if not os.path.isfile(fname_wtd):
        download_flag = True
        break
if download_flag:
    start_date_str = datetime_dim[0].strftime('%Y-%m-%d')
    end_date_str = (datetime_dim[len(datetime_dim)-1]+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    options_wtd = {"dataset": "conus1_baseline_mod", "variable": "water_table_depth", "temporal_resolution": "daily", "start_time": start_date_str, "end_time": end_date_str, "grid_bounds":[grid_minx,grid_miny,grid_maxx,grid_maxy]}  
    hf_data = hf.get_gridded_data(options_wtd)
    if hf_data.shape[0] != int((end_datetime - start_datetime).days) + 1: sys.exit('ERROR hydroframe returned data of unexpected time length.')
    hf_conus1grid_temp = numpy.empty((1888,3342))
    for i in range(len(datetime_dim)):
        idatetime = datetime_dim[i]
        hf_conus1grid_temp[grid_miny:grid_maxy,grid_minx:grid_maxx] = hf_data[i,:,:]
        memfile = rasterio.io.MemoryFile()
        hf_conus1data = memfile.open(driver = "GTiff", height = hf_conus1grid_temp.shape[0], width = hf_conus1grid_temp.shape[1], crs=conus1_proj, transform = conus1_transform, nodata = numpy.nan, count = 1, dtype = numpy.float64)
        hf_conus1data.write(hf_conus1grid_temp,1)
        fname = os.path.join(ProjectWTDRawDataDirectory,'wtd_'+idatetime.strftime('%Y%m%d')+'.tiff')
        wtd_data, wtd_transform = rasterio.mask.mask(hf_conus1data, buffered_domain.to_crs(hf_conus1data.crs), crop=True, all_touched=True, filled =True, nodata = numpy.nan)
        wtd_meta = hf_conus1data.meta
        wtd_meta.update({"driver": "GTiff","height": wtd_data.shape[1],"width": wtd_data.shape[2],"transform": wtd_transform, "nodata" : numpy.nan})
        with rasterio.open(fname,'w',**wtd_meta) as wtd_dataset:
            wtd_dataset.write(wtd_data[0,:,:],1)

###### Increase resolution of ParFlow grid to 2 m (from 1000 m)
###### Maybe just do this on the fly, in-memory, per time step, after putting the topographic info into this grid using one instance of the wtd grid

In [13]:
#resample_methods = {'nearest':[rasterio.enums.Resampling.nearest,ProjectWTDNearestDataDirectory],
#                    'bilinear':[rasterio.enums.Resampling.bilinear,ProjectWTDBilinearDataDirectory]}
resample_methods = {'bilinear':[rasterio.enums.Resampling.bilinear,ProjectWTDBilinearDataDirectory]}
for resample_method_name, [resample_method, resample_dir] in resample_methods.items():
    for idatetime in datetime_dim:
        fname_wtd = os.path.join(ProjectWTDRawDataDirectory,'wtd_'+idatetime.strftime('%Y%m%d')+'.tiff')
        fname_wtd_hr = os.path.join(resample_dir,'wtd_'+idatetime.strftime('%Y%m%d')+'_highresolution_'+resample_method_name+'.tiff')
        if not os.path.isfile(fname_wtd_hr) or OVERWRITE_FLAG:
            with rasterio.open(fname_wtd) as wtd_dataset:
                wtd_data_hr = wtd_dataset.read(out_shape=(wtd_dataset.count,int(wtd_dataset.height * 500),int(wtd_dataset.width * 500)),resampling=resample_method)
                wtd_transform_hr = wtd_dataset.transform * wtd_dataset.transform.scale((wtd_dataset.width / wtd_data_hr.shape[-1]),(wtd_dataset.height / wtd_data_hr.shape[-2]))
                wtd_meta_hr = wtd_dataset.meta
                wtd_meta_hr.update({"driver": "GTiff","height": wtd_data_hr.shape[1],"width": wtd_data_hr.shape[2],"transform": wtd_transform_hr, "nodata" : numpy.nan})
                with rasterio.open(fname_wtd_hr, "w", **wtd_meta_hr) as wtd_dataset_hr:
                    wtd_dataset_hr.write(wtd_data_hr)

###### Create domain mask

In [14]:
fname_domain_mask = os.path.join(ProjectDataDirectory,"domain_mask.tiff")
if not os.path.isfile(fname_domain_mask) or OVERWRITE_FLAG:
    fname_wtd_hr = os.path.join(ProjectWTDDataDirectory,'wtd_'+datetime_dim[0].strftime('%Y%m%d')+'_highresolution_bilinear_resample.tiff') # point to bilinear but all resampling methods should share the same grid / resolution
    with rasterio.open(fname_wtd_hr,'r') as wtd_highres:
        domain_data = wtd_highres.read(1)
        domain_meta = wtd_highres.meta
        domain_crs = wtd_highres.crs
        domain_transform = wtd_highres.transform
        domain_mask = rasterio.features.rasterize(shapes=domain.to_crs(domain_crs)['geometry'],out_shape=domain_data.shape,transform=domain_transform,fill=0,all_touched=True,dtype=rasterio.uint8,default_value=1)
        domain_meta.update({"driver": "GTiff","height": domain_data.shape[0],"width": domain_data.shape[1],"transform": domain_transform,"dtype": rasterio.uint8,"nodata":0})
        with rasterio.open(fname_domain_mask, 'w', **domain_meta) as dst:
            dst.write(domain_mask,indexes=1)

###### Get soil texture data
###### NOTE: uses pysda via https://github.com/ncss-tech/pysda.git -- pysdaDir must be set as path to pysda clone

In [15]:
fname_soil_texture = os.path.join(ProjectSoilsDataDirectory,"soil_texture.gpkg")
fname_soil_transmissivity_decay_factor = os.path.join(ProjectSoilsDataDirectory,"soil_transmissivity_decay_factor.tiff")
if not os.path.isfile(fname_soil_transmissivity_decay_factor) or OVERWRITE_FLAG:
    if pysdaDir not in sys.path: sys.path.append(pysdaDir)
    import sdapoly, sdaprop, sdainterp
    soils_aoi = sdapoly.gdf(domain)
    sandtotal_r=sdaprop.getprop(df=soils_aoi,column='mukey',method='dom_comp_num',top=0,bottom=400,prop='sandtotal_r',minmax=None,prnt=False,meta=False)
    claytotal_r=sdaprop.getprop(df=soils_aoi,column='mukey',method='dom_comp_num',top=0,bottom=400,prop='claytotal_r',minmax=None,prnt=False,meta=False)
    def calc_texture(row): 
        try: sand = float(row['sandtotal_r'])
        except: return 'None'
        try: clay = float(row['claytotal_r'])
        except: return 'None'
        return soiltexture.getTexture(sand,clay)
    def calc_f(row):
        if row['texture'] in transmissivity_decay_factor: return transmissivity_decay_factor[row['texture']]
        else: 
            print('WARNING: Could not find transmissivity decay factor for soil texture ',row['texture'])
            return numpy.mean(list(transmissivity_decay_factor.values()))
    fname_wtd_hr = os.path.join(ProjectWTDBilinearDataDirectory,'wtd_'+datetime_dim[0].strftime('%Y%m%d')+'_highresolution_bilinear.tiff') # point to bilinear but all resampling methods should share the same grid / resolution
    with rasterio.open(fname_wtd_hr,'r') as wtd_highres:
        dummy_meta = wtd_highres.meta
        dummy_data = wtd_highres.read(1)
        soils_aoi = soils_aoi.merge(pandas.merge(sandtotal_r,claytotal_r,on='mukey'),on='mukey')
        soils_aoi['texture'] = soils_aoi.apply(calc_texture, axis=1)
        soils_aoi['f'] = soils_aoi.apply(calc_f, axis=1)
        soils_aoi = soils_aoi.to_crs(wtd_highres.crs)
        soils_aoi.to_crs('EPSG:4326').to_file(fname_soil_texture, driver="GPKG")
        soils_shapes = ((geom,value) for geom, value in zip(soils_aoi.geometry, soils_aoi['f']))
        texture_data = rasterio.features.rasterize(shapes=soils_shapes,out_shape=dummy_data.shape,transform=wtd_highres.transform,fill=numpy.nan,all_touched=True,dtype=rasterio.float32,default_value=numpy.nan)
        dummy_meta.update({"driver": "GTiff","height": dummy_data.shape[0],"width": dummy_data.shape[1],"transform": wtd_highres.transform,"dtype": rasterio.float32,"nodata":numpy.nan})
        with rasterio.open(fname_soil_transmissivity_decay_factor, 'w', **dummy_meta) as dst:
            dst.write(texture_data,indexes=1)

###### Get DEM
###### NOTE: py3dep.check_3dep_availability may show availability of 1 meter resolution DEM but py3dep.get_dem does not work if called with resolution < 10, in my experience

In [16]:
fname_dem_original = os.path.join(ProjectDEMDataDirectory,"dem_original.tif")
if not os.path.isfile(fname_dem_original) or OVERWRITE_FLAG:
    if os.path.isfile(DEMfname):
        shutil.copy2(DEMfname,fname_dem_original)
    else:
        #dtavailability = py3dep.check_3dep_availability([lon_min,lat_min,lon_max,lat_max]) # can see dem resolution availability here
        dem_original = py3dep.get_dem(geometry=[lon_min,lat_min,lon_max,lat_max],resolution=10) # wpn't work with < 10 m
        dem_original.rio.to_raster(fname_dem_original)
        del dem_original

###### Reproject DEM to match resampled (high resolution) Parflow grid

In [17]:
fname_dem = os.path.join(ProjectDEMDataDirectory,"dem_reprojected.tif")
if not os.path.isfile(fname_dem) or OVERWRITE_FLAG:
    fname_wtd_hr = os.path.join(ProjectWTDDataDirectory,'wtd_'+datetime_dim[0].strftime('%Y%m%d')+'_highresolution_bilinear_resample.tiff') # point to bilinear but all resampling methods should share the same grid / resolution
    with rasterio.open(fname_dem_original,'r') as dem_original:
        dem_reprojected_data, dem_reprojected_transform = rasterio.warp.reproject(
                source = dem_original.read(1),
                destination = rasterio.open(fname_wtd_hr,'r').read(1),
                src_transform=dem_original.transform,
                src_crs=dem_original.crs,
                dst_transform=rasterio.open(fname_wtd_hr,'r').transform,
                dst_crs=rasterio.open(fname_wtd_hr,'r').crs,
                resampling=rasterio.enums.Resampling.bilinear)
        dem_reprojected_data = dem_reprojected_data[0,:,:]
        dem_reprojected_data = numpy.where(domain_mask==1,dem_reprojected_data,numpy.nan)
        dem_reprojected_meta = rasterio.open(fname_wtd_hr,'r').meta
        dem_reprojected_meta.update({"driver": "GTiff","height": dem_reprojected_data.shape[0],"width": dem_reprojected_data.shape[1],"transform": dem_reprojected_transform, "nodata":numpy.nan})
        with rasterio.open(fname_dem, "w", **dem_reprojected_meta) as dem_reprojected:
            dem_reprojected.write(dem_reprojected_data,1)

###### Initialize whitebox environment

In [18]:
wbe = whitebox_workflows.WbEnvironment()

###### Breach the DEM (minimally invasive alternative to filling the DEM?) (see https://www.whiteboxgeo.com/manual/wbt_book/available_tools/hydrological_analysis.html#BreachDepressionsLeastCost)

In [19]:
fname_dem_breached = os.path.join(ProjectDEMDataDirectory,"dem_reprojected_breached.tif")
if not os.path.isfile(fname_dem_breached) or OVERWRITE_FLAG:
    dem_wbe = wbe.read_raster(fname_dem)
    dem_breached_wbe = wbe.breach_depressions_least_cost(dem=dem_wbe)
    wbe.write_raster(dem_breached_wbe, fname_dem_breached, compress=False)
    del dem_wbe, dem_breached_wbe

###### Calculate flow accumulation / watershed area (required for TWI calculation) using d-infinity (https://www.whiteboxgeo.com/manual/wbt_book/available_tools/hydrological_analysis.html#DInfFlowAccumulation)

In [20]:
fname_acc = os.path.join(ProjectDEMDataDirectory,"flow_acc.tif")
if not os.path.isfile(fname_acc) or OVERWRITE_FLAG:
    dem_breached_wbe = wbe.read_raster(fname_dem_breached)
    acc_wbe = wbe.dinf_flow_accum(dem=dem_breached_wbe,out_type='sca',log_transform=False)                                         
    wbe.write_raster(acc_wbe, fname_acc, compress=False)
    del dem_breached_wbe, acc_wbe

###### Calculate slope (required for TWI calculation)

In [21]:
fname_slope = os.path.join(ProjectDEMDataDirectory,"slope.tif")
if not os.path.isfile(fname_slope) or OVERWRITE_FLAG:
    dem_wbe = wbe.read_raster(fname_dem_breached)
    slp_wbe = wbe.slope(dem=dem_wbe)   
    wbe.write_raster(slp_wbe, fname_slope, compress=False)
    del dem_wbe, slp_wbe

###### Calculate TWI

In [22]:
fname_twi = os.path.join(ProjectTWIDataDirectory,"twi.tif")
if not os.path.isfile(fname_twi) or OVERWRITE_FLAG:
    acc_wbe = wbe.read_raster(fname_acc)
    slp_wbe = wbe.read_raster(fname_slope)
    twi_wbe = wbe.wetness_index(specific_catchment_area=acc_wbe,slope=slp_wbe)
    wbe.write_raster(twi_wbe, fname_twi, compress=False)
    del acc_wbe, slp_wbe, twi_wbe

###### Upsample TWI to caculate mean TWI per Parflow grid cell 
###### TODO: change to moving window average TWI? Each pixel will have a mean TWI for it's unique window? Currently biased for pixels on border of Parflow grid window?

In [23]:
fname_twi_upsample = os.path.join(ProjectTWIDataDirectory,"twi_upsample.tif")
if not os.path.isfile(fname_twi_upsample) or OVERWRITE_FLAG:
    with rasterio.open(fname_twi,'r') as twi_dataset:
        twi_upsample_data = twi_dataset.read(out_shape=(twi_dataset.count,int(twi_dataset.height / 100),int(twi_dataset.width / 100)),resampling=rasterio.enums.Resampling.average)
        twi_upsample_transform = twi_dataset.transform * twi_dataset.transform.scale((twi_dataset.width / twi_upsample_data.shape[-1]),(twi_dataset.height / twi_upsample_data.shape[-2]))
        twi_upsample_meta = twi_dataset.meta
        twi_upsample_meta.update({"driver": "GTiff","height": twi_upsample_data.shape[1],"width": twi_upsample_data.shape[2],"transform": twi_upsample_transform})
        with rasterio.open(fname_twi_upsample, "w", **twi_upsample_meta) as twi_upsample_dataset:
            twi_upsample_dataset.write(twi_upsample_data)

###### Downsample average Parflow grid cell TWI to high resolution ParFlow grid (to enable vectorized calculations of inundated area)

In [24]:
fname_twi_downsample = os.path.join(ProjectTWIDataDirectory,"twi_downsample.tif")
if not os.path.isfile(fname_twi_downsample) or OVERWRITE_FLAG:
    with rasterio.open(fname_twi_upsample,'r') as twi_upsample_dataset:
        twi_downsample_data = twi_upsample_dataset.read(out_shape=(twi_upsample_dataset.count,int(twi_upsample_dataset.height * 100),int(twi_upsample_dataset.width * 100)),resampling=rasterio.enums.Resampling.nearest)
        twi_downsample_transform = twi_upsample_dataset.transform * twi_upsample_dataset.transform.scale((twi_upsample_dataset.width / twi_downsample_data.shape[-1]),(twi_upsample_dataset.height / twi_downsample_data.shape[-2]))
        twi_downsample_meta = twi_upsample_dataset.meta
        twi_downsample_meta.update({"driver": "GTiff","height": twi_downsample_data.shape[1],"width": twi_downsample_data.shape[2],"transform": twi_downsample_transform})
        with rasterio.open(fname_twi_downsample, "w", **twi_downsample_meta) as twi_downsample_dataset:
            twi_downsample_dataset.write(twi_downsample_data)

###### Define function to perform inundation calculation

In [25]:
def calculate_indundation(wtd_mean,twi_local,twi_mean,f,domain_mask):
    """Calculate inundation using Equation 3 of Zhang et al. (2016) (https://doi.org/10.5194/bg-13-1387-2016).
    Arguments:
        wtd_mean:    grid of mean water table depth values  (zeta sub m in equation 3 of Zhang et al.) 
        twi_local:   grid of local TWI values               (lambda sub l in equation 3 of Zhang et al.)
        twi_mean:    grid of mean TWI values                (lamda sub m in equation 3 of Zhang et al.)
        f:           grid of f parameter values             (f in equation 3 and table 1 of Zhang et al.)
        domain_mask: grid of model domain                   (1=domain,0=not domain)
    Returns:
        wtd_local:   grid of local water table depth values (zeta sub l in equation 3 of Zhang et al.)
    """
    wtd_mean  = wtd_mean*(-1)                                                             # values must be negative so multiply by -1
    wtd_local = numpy.where(domain_mask==1,(1/f)*(twi_local-twi_mean)+wtd_mean,numpy.nan) # calculate local water depth using equation 3 from Zhang et al. (2016) where domain_mask=1 (i.e., within the model domain), otherise give a NaN value 
    wtd_local = numpy.where(wtd_local>=0,1,numpy.nan)                                     # give value of 1 where local water table depth is >= 0 (i.e. at or above the surface), otherwise give a NaN value
    return wtd_local

###### Do the inundated area calculations

In [26]:
twi_local = rasterio.open(fname_twi,'r').read(1)
twi_mean = rasterio.open(fname_twi_downsample,'r').read(1)
domain_mask = rasterio.open(fname_domain_mask,'r').read(1)
transmissivity_decay_factor = rasterio.open(fname_soil_transmissivity_decay_factor,'r').read(1)
dtp = {'bilinear':ProjectWTDBilinearDataDirectory,'nearest':ProjectWTDNearestDataDirectory}
for idatetime in datetime_dim:
    for method, dir_wtd in dtp.items():
        fname_wtd_mean = os.path.join(dir_wtd,'wtd_'+idatetime.strftime('%Y%m%d')+'_highresolution_'+method+'.tiff')
        fname_output = os.path.join(ProjectOutputDirectory,'inundatedarea_'+idatetime.strftime('%Y%m%d')+'_'+method+'.tiff')
        if os.path.isfile(fname_wtd_mean):
            with rasterio.open(fname_wtd_mean,'r') as wtd_mean_dataset:
                wtd_mean = wtd_mean_dataset.read(1)
                wtd_local = calculate_indundation(wtd_mean,twi_local,twi_mean,transmissivity_decay_factor,domain_mask)
                with rasterio.open(fname_output, "w", **wtd_mean_dataset.meta) as wtd_local_dataset:
                    wtd_local_dataset.write(wtd_local,1)
del twi_local, twi_mean, domain_mask

###### Create summary rasters

In [27]:
#for method in ['nearest','bilinear','cubic']:
domain_mask = rasterio.open(fname_domain_mask,'r').read(1)
for method in ['bilinear']:
    fname_output = os.path.join(ProjectOutputDirectory,'summary_grid_'+datetime_dim[0].strftime('%Y%m%d')+'_to_'+datetime_dim[len(datetime_dim)-1].strftime('%Y%m%d')+'_'+method+'.tiff')
    count_grid = -1
    for idatetime in datetime_dim:
        fname_wtd_local = os.path.join(ProjectOutputDirectory,'inundatedarea_'+idatetime.strftime('%Y%m%d')+'_'+method+'.tiff') 
        inundation_data = rasterio.open(fname_wtd_local,'r').read(1)
        if isinstance(count_grid,int): # init count_grid if start date
            count_grid = numpy.zeros(inundation_data.shape)
            count_grid = numpy.where(domain_mask==1,count_grid,numpy.nan)
        count_grid += numpy.where(inundation_data==1,1,0)
    count_grid = numpy.where(count_grid==0,numpy.nan,count_grid) # convert to percent time that each cell is inundated
    with rasterio.open(fname_output, "w", **rasterio.open(fname_wtd_local,'r').meta) as summary_dataset:
        summary_dataset.write(count_grid,1)
del domain_mask